## Revise annotation from bad list

* Pre-porcess the CSV list firt, then usd full_good_list_csv to copy all-good image to train_augm/images for flip
* todo: move the image copy to end of processing.

In [1]:
import os
import cv2
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

import shutil

from tqdm import tqdm

In [2]:
"""
# Now you can use `progress_apply` instead of `apply`
# and `progress_map` instead of `map`
"""
tqdm.pandas()

### Load 1st. (deafult) annotation of training set

* 1000 samples

In [3]:
ann = 'train/annotation.csv'
df = pd.read_csv(ann)
df.head(5)

,images,x1,y1,x2,y2
0,image_0001.png,55,136,75,383
1,image_0002.png,62,139,42,385
2,image_0003.png,47,117,76,368
3,image_0004.png,66,119,37,376
4,image_0005.png,58,147,85,381


In [4]:
df.describe()

,x1,y1,x2,y2
count,1000.000000,1000.000000,1000.00000,1000.000000
mean,57.679000,133.471000,54.78000,377.934000
std,8.145693,14.154629,24.19804,7.757049
min,24.000000,99.000000,10.00000,334.000000
25%,53.000000,122.750000,31.00000,375.000000
50%,58.000000,136.000000,53.50000,381.000000
75%,62.250000,142.000000,78.00000,383.000000
max,84.000000,182.000000,96.00000,387.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
images    1000 non-null object
x1        1000 non-null int64
y1        1000 non-null int64
x2        1000 non-null int64
y2        1000 non-null int64
dtypes: int64(4), object(1)
memory usage: 39.2+ KB


In [6]:
df.columns

Index(['images', 'x1', 'y1', 'x2', 'y2'], dtype='object')

In [7]:
df.shape

(1000, 5)

In [8]:
df.index

RangeIndex(start=0, stop=1000, step=1)

In [9]:
df.iloc[0:2,:]

,images,x1,y1,x2,y2
0,image_0001.png,55,136,75,383
1,image_0002.png,62,139,42,385


In [10]:
# f'Text {variable} : [] {variable}' == 'Text {} : [] {}'.format(variable, variable)

print(f'df list size : {len(df)}, Index: {df.index}')

df_index = 0
print(f'Image: {df.images[df_index]} x1:{df.x1[df_index]} y1:{df.y1[df_index]} x1:{df.x2[df_index]} y1:{df.y2[df_index]}')

df list size : 1000, Index: RangeIndex(start=0, stop=1000, step=1)
Image: image_0001.png x1:55 y1:136 x1:75 y1:383


In [11]:
# check y1 range
df[df.y1 > 170]

,images,x1,y1,x2,y2
183,image_0184.jpg,48,173,31,382
184,image_0185.jpg,64,175,86,383
185,image_0186.jpg,55,173,26,381
574,image_0575.png,55,178,10,379
662,image_0663.jpg,48,182,53,383
970,image_0971.png,49,171,26,382


In [12]:
# df

### Load 2nd annotation (1/6 424 addon)


* train_20210106
- 424 images, from image_0001.png to image_0424.jpg
* todo
* rename image_6001.png to image_6424.jpg in CSV and train_20210106_a6
*

In [13]:
!mkdir train_20210106_a6

mkdir: cannot create directory ‘train_20210106_a6’: File exists


In [14]:
!ls -l train_20210106_a6/ |wc -l

425


In [15]:
a6_csv = 'train_20210106/annotation_20210106.csv'
a6_df = pd.read_csv(a6_csv)
len(a6_df)

424

In [16]:
a6_df.head(5)

,images,x1,y1,x2,y2
0,image_0001.png,56,119,82,383
1,image_0002.png,57,132,37,378
2,image_0003.png,61,175,81,384
3,image_0004.png,53,179,32,381
4,image_0005.png,50,120,81,383


In [17]:
for i, im_name in enumerate(a6_df['images']):
    if i < 10:
        print('{}'.format(im_name))

image_0001.png
image_0002.png
image_0003.png
image_0004.png
image_0005.png
image_0006.png
image_0007.png
image_0008.png
image_0009.png
image_0010.png


### change element value inline of DataFrame

* https://zhuanlan.zhihu.com/p/100064394 Pandas數據處理三板斧——map、apply、applymap詳解
* #①使用字典进行映射
data["gender"] = data["gender"].map({"男":1, "女":0})
​
#②使用函数
def gender_map(x):
    gender = 1 if x == "男" else 0
    return gender
#注意这里传入的是函数名，不带括号
data["gender"] = data["gender"].map(gender_map)
*

#### Try how to change element by map()

In [18]:
old_name = 'image_0021.png'
str_1, str_2 = old_name.split('_',-1);print('str_2 : {}'.format(str_2))
str_3, str_4 = str_2.split('.',-1);print('str_2 : {} . {}'.format(str_3,str_4))

a6_num = int(str_3) + int(6000)
print('a6_num: {}'.format(a6_num))

new_a6_name = 'image_' + str(a6_num) + '.' + str_4 
print('new_a6_name: {}'.format(new_a6_name))


str_2 : 0021.png
str_2 : 0021 . png
a6_num: 6021
new_a6_name: image_6021.png


### Change image number (add 6k) and copy to 'train_20210106_a6/'
#### merge into fun

In [19]:
#os.rename(filename, filename[:6]+'/'+filename[6:])
# os.rename will move the source file to new file path.
# shutil.copyfile(s, d) maybe better

In [20]:
def add_6000_map(img_name):
    str_1, str_2 = img_name.split('_',-1)#;print('str_2 : {}'.format(str_2))
    str_3, str_4 = str_2.split('.',-1)#;print('str_2 : {} . {}'.format(str_3,str_4))

    a6_num = int(str_3) + int(6000)
    #print('a6_num: {}'.format(a6_num))

    new_a6_name = 'image_' + str(a6_num) + '.' + str_4 
    print('new_a6_name: {}\t <--- {}'.format(new_a6_name, img_name))
    
    
    old_60k_path = 'train_20210106/images/' + img_name
    new_60k_path = 'train_20210106_a6/' + new_a6_name   
    
    #NOTE: 'os.rename' will move the old path/file to new_path/file!!!
    #os.rename(old_60k_path, new_60k_path)
    shutil.copyfile(old_60k_path, new_60k_path)
    print('new_a6_name: {}\t <--- {}'.format(new_a6_name, img_name))
    
    return new_a6_name

In [21]:
a6_df["images"] = a6_df["images"].map(add_6000_map)

new_a6_name: image_6001.png	 <--- image_0001.png
new_a6_name: image_6001.png	 <--- image_0001.png
new_a6_name: image_6002.png	 <--- image_0002.png
new_a6_name: image_6002.png	 <--- image_0002.png
new_a6_name: image_6003.png	 <--- image_0003.png
new_a6_name: image_6003.png	 <--- image_0003.png
new_a6_name: image_6004.png	 <--- image_0004.png
new_a6_name: image_6004.png	 <--- image_0004.png
new_a6_name: image_6005.png	 <--- image_0005.png
new_a6_name: image_6005.png	 <--- image_0005.png
new_a6_name: image_6006.png	 <--- image_0006.png
new_a6_name: image_6006.png	 <--- image_0006.png
new_a6_name: image_6007.png	 <--- image_0007.png
new_a6_name: image_6007.png	 <--- image_0007.png
new_a6_name: image_6008.png	 <--- image_0008.png
new_a6_name: image_6008.png	 <--- image_0008.png
new_a6_name: image_6009.png	 <--- image_0009.png
new_a6_name: image_6009.png	 <--- image_0009.png
new_a6_name: image_6010.png	 <--- image_0010.png
new_a6_name: image_6010.png	 <--- image_0010.png
new_a6_name: image_6

In [22]:
!ls train_20210106/images/image_0001.png

train_20210106/images/image_0001.png


In [23]:
# check new 6k numbers
!ls train_20210106_a6/image_6001.png

train_20210106_a6/image_6001.png


In [24]:
len(a6_df)

424

In [25]:
a6_df.head(5)

,images,x1,y1,x2,y2
0,image_6001.png,56,119,82,383
1,image_6002.png,57,132,37,378
2,image_6003.png,61,175,81,384
3,image_6004.png,53,179,32,381
4,image_6005.png,50,120,81,383


In [26]:
a6_df.to_csv('annotation_a6_df.csv', index=False)

#### Append new 2nd to 1st data list

In [27]:
a6 = 'annotation_a6_df.csv'
a6_df = pd.read_csv(a6)


df_merge = pd.concat([df, a6_df])

In [28]:
df_merge.to_csv('annotation_1424_merge.csv', index=False)

In [29]:
len(df)

1000

In [30]:
len(df_merge)

1424

### Copy a6 image to train/images

* cp train_20210106_a6/* to train/images/ for latter remove bad samples

In [31]:
!cp train_20210106_a6/image_6* train/images/

In [32]:
# should be all samples (1000 + 424 images)
!ls -l train/images/image_* | wc -l

1424


## Plot ALL Samples for search bad annotation

In [ ]:
ann_1424 = 'annotation_1424_merge.csv'
df_1424= pd.read_csv(ann_1424)
len(df_1424)

In [ ]:
"""
Here the df_merge list for 1~1000 and 6001~6004,
but we are give it as index, so need -5001 for 6a samples.

1424/30 ~ 48

"""
print("Plot ALL Samples for search bad annotation: ", len(df_1424))

step_num = 48
block_num = 30

for i_s in range(step_num):
    print(i_s)
    
    plt.figure(figsize=(20, 20))
    fig_name = 'review_fig/' + str(i_s) + '.png'
    for i in range(block_num):
    #     print(i)
        if i > 1000:
            ax = plt.subplot(3, 10, int(i+1))
            plt_ud_point_multiple(df_1424,i-5001 + i_s*block_num)
        else:
            ax = plt.subplot(3, 10, int(i+1))
            plt_ud_point_multiple(df_1424,i + i_s*block_num)

    # SAVE 30 images in one figure block #
#     plt.savefig(fig_name, bbox_inches='tight', pad_inches=0)

### List of bad samples

In [ ]:
#bad smaples
1.點不在熱圖上: 
67/ 170/ 235/ 245/ 313/ 373/ 410/ 478/ 625/ 637/ 656/ 662/ 801/ 826/ 872/ 891/ 933/ 953
2.點極端偏左偏右不在長軸上: 
749/ 589/ 751/ 609/ 404/ 649/ 797/ 685/ 83/ 171/ 600
圖片經確認，請皆忽略不使用（+image_0980.png）

total:30 bad samples

副檔名可能為jpg/JPG or png/PNG 需要自行檢查


https://stackoverflow.com/questions/46715379/remove-rows-of-dataframe-of-values-in-a-list-present-in-another-list

### Check bad sampls

* plot each bad samples for double check.

In [33]:
df_merge = 'annotation_1424_merge.csv'
df_merge= pd.read_csv(df_merge)
len(df_merge)

1424

In [34]:
ind = 982
plt_ud_point(df_merge,ind -1)

NameError: name 'plt_ud_point' is not defined

In [ ]:
# for 6a image, 6069 --> 1069 for index #
# or 69 with +999 for 6a's 6069
ind = 420
plt_ud_point(df_merge,ind + 999)

In [ ]:
"""
Here the df_merge list for 1~1000 and 6001~6004,
but we are give it as index, so need -5001 for 6a samples.
"""
print("bad_sample_list : ", len(bad_sample_list))
plt.figure(figsize=(18, 35))
for i, ele in enumerate(bad_sample_list):
#     print(i, ele)
    if ele > 1000:
        ax = plt.subplot(8, 10, int(i+1))
        plt_ud_point_multiple(df_merge,ele-5001)
    else:
        ax = plt.subplot(8, 10, int(i+1))
        plt_ud_point_multiple(df_merge,ele-1)

In [ ]:
"""
依照前人找出的類型，應該還有更多類似標註存在需要被從訓練集中刪除。
"""

### Create bad sample list

In [ ]:
# Fist bad list
# bad_sample_list=[67, 170, 235, 245, 313, 373, 410, 478, 625, 637, 656, 662, 801, 826, 872, 891, 933, 953, 749, 589, 751, 609, 404, 649, 797, 685, 83, 171, 600, 980]
# bad_sample_list[:5]

# # 2nd bas list
# bad_sample_list=[96,
#                  136,
#                  202,
#                  226,
#                  227,
#                  262,
#                  357,
#                  413,
#                  471,
#                  477,
#                  497,
#                  481,
#                  485,
#                  486,
#                  520,
#                  598,
#                  623,
#                  624,
#                  633,
#                  639,
#                  647,
#                  654,
#                  663,
#                  692,
#                  699,
#                  719,
#                  811,
#                  818,
#                  886,
#                  888,
#                  910,
#                  968,
#                  982,
#                  6077,
#                  6078,
#                  6079,
#                  6091,
#                  6093,6094,6111,6115,6162,6168,6170,6281,6349,6353,6359,6420
#                 ]


# 2nd bas list 30+ 49 = 79 -> 1345 current best
# bad_sample_list=[67, 170, 235, 245, 313, 373, 410, 478, 625, 637, 656, 662, 801, 826, 
#                 872, 891, 933, 953, 749, 589, 751, 609, 404, 649, 797, 685, 83, 171, 600, 980,
#                 96,
#                  136,
#                  202,
#                  226,
#                  227,
#                  262,
#                  357,
#                  413,
#                  471,
#                  477,
#                  497,
#                  481,
#                  485,
#                  486,
#                  520,
#                  598,
#                  623,
#                  624,
#                  633,
#                  639,
#                  647,
#                  654,
#                  663,
#                  692,
#                  699,
#                  719,
#                  811,
#                  818,
#                  886,
#                  888,
#                  910,
#                  968,
#                  982,
#                  6077,
#                  6078,
#                  6079,
#                  6091,
#                  6093,6094,6111,6115,6162,6168,6170,6281,6349,6353,6359,6420
#                 ]


# # 2nd bas list 30+ 49 = 79 -> 1345 current best [re-ordering] 404,413,589, is good. 6281
# bad_sample_list=[67, 83, 96, 170, 171, 235, 245, 313, 373, 404, 410, 478, 
#                  589, 600, 609, 625, 637, 649, 656, 662, 
#                  685, 749, 751, 797, 801, 826, 872, 891, 933, 953, 980,
#                  136,
#                  202,
#                  226,
#                  227,
#                  262,
#                  357,
#                  413,
#                  471,
#                  477,
#                  497,
#                  481,
#                  485,
#                  486,
#                  520,
#                  598,
#                  623,
#                  624,
#                  633,
#                  639,
#                  647,
#                  654,
#                  663,
#                  692,
#                  699,
#                  719,
#                  811,
#                  818,
#                  886,
#                  888,
#                  910,
#                  968,
#                  982,
#                  6077,
#                  6078,
#                  6079,
#                  6091,
#                  6093,6094,6111,6115,6162,6168,6170,6281,6349,6353,6359,6420
#                 ]


# bad_sample_list[:5]

In [ ]:
# # 2020-01-26
# # HPL's v1 bad list
# # 301
# bad_sample_list=[9,
# 6,
# 7,
# 21,
# 49,
# 50,
# 51,
# 55,
# 57,
# 60,
# 61,
# 63,
# 67,
# 73,
# 74,
# 75,
# 82,
# 83,
# 85,
# 86,
# 95,
# 96,
# 103,
# 122,
# 136,
# 139,
# 143,
# 144,
# 145,
# 147,
# 148,
# 159,
# 165,
# 166,
# 170,
# 171,
# 173,
# 177,
# 179,
# 183,
# 187,
# 193,
# 195,
# 198,
# 199,
# 202,
# 203,
# 207,
# 225,
# 227,
# 228,
# 230,
# 232,
# 235,
# 236,
# 238,
# 245,
# 252,
# 254,
# 257,
# 261,
# 262,
# 265,
# 267,
# 269,
# 275,
# 276,
# 278,
# 283,
# 291,
# 303,
# 305,
# 313,
# 315,
# 319,
# 321,
# 324,
# 335,
# 337,
# 341,
# 343,
# 354,
# 357,
# 361,
# 363,
# 364,
# 365,
# 369,
# 373,
# 377,
# 393,
# 399,
# 404,
# 407,
# 410,
# 414,
# 425,
# 433,
# 442,
# 445,
# 451,
# 462,
# 467,
# 468,
# 471,
# 477,
# 478,
# 479,
# 480,
# 481,
# 482,
# 484,
# 485,
# 486,
# 487,
# 488,
# 497,
# 498,
# 509,
# 510,
# 517,
# 518,
# 526,
# 527,
# 551,
# 552,
# 553,
# 575,
# 580,
# 588,
# 589,
# 598,
# 599,
# 600,
# 601,
# 607,
# 609,
# 610,
# 611,
# 612,
# 616,
# 617,
# 618,
# 619,
# 621,
# 622,
# 623,
# 624,
# 625,
# 628,
# 629,
# 630,
# 633,
# 636,
# 637,
# 638,
# 646,
# 647,
# 648,
# 649,
# 653,
# 654,
# 655,
# 656,
# 658,
# 659,
# 661,
# 662,
# 663,
# 678,
# 678,
# 685,
# 688,
# 690,
# 698,
# 699,
# 700,701,
# 708,
# 709,
# 712,
# 719,
# 734,
# 737,
# 742,
# 744,
# 745,
# 746,
# 747,
# 748,
# 749,
# 751,
# 752,
# 754,
# 760,
# 761,
# 762,
# 763,
# 765,
# 766,
# 767,
# 768,
# 769,
# 785,
# 787,
# 788,
# 789,
# 797,
# 801,
# 804,
# 805,
# 808,
# 810,
# 811,
# 814,
# 816,
# 818,
# 826,
# 832,
# 848,
# 855,
# 872,
# 874,
# 877,
# 878,
# 882,
# 884,
# 886,
# 891,
# 895,
# 902,
# 910,
# 917,
# 930,
# 933,
# 937,
# 944,
# 951,
# 952,
# 962,
# 968,
# 972,
# 974,
# 980,
# 982,
# 985,
# 989,
# 990,
# 6006,
# 6021,
# 6024,
# 6031,
# 6032,
# 6037,
# 6049,
# 6053,
# 6060,
# 6063,
# 6075,
# 6079,
# 6083,
# 6091,
# 6093,
# 6094,
# 6098,
# 6105,
# 6111,
# 6115,
# 6117,
# 6127,
# 6128,
# 6133,
# 6134,
# 6162,
# 6168,
# 6169,
# 6170,
# 6183,
# 6184,
# 6195,
# 6203,
# 6205,
# 6220,
# 6240,
# 6246,
# 6281,
# 6287,
# 6299,
# 6317,
# 6318,
# 6321,
# 6327,
# 6329,
# 6330,
# 6331,
# 6349,
# 6359,
# 6365,
# 6371,
# 6372,
# 6406,
# 6420
#                 ]

In [41]:
# # HPLv2 TOE
# bad_sample_list=[50,
# 61,
# 67,
# 75,
# 83,
# 96,
# 122,
# 136,
# 147,
# 170,
# 171,
# 173,
# 198,
# 202,
# 226,
# 227,
# 228,
# 235,
# 245,
# 262,
# 265,
# 275,
# 278,
# 291,
# 303,
# 313,
# 324,
# 335,
# 357,
# 369,
# 373,
# 410,
# 451,
# 467,
# 471,
# 477,
# 478,
# 479,
# 485,
# 520,
# 526,
# 552,
# 580,
# 598,
# 600,
# 609,
# 610,
# 616,
# 617,
# 621,
# 623,
# 624,
# 625,
# 633,
# 637,
# 647,
# 649,
# 653,
# 654,
# 655,
# 656,
# 662,
# 663,
# 685,
# 699,
# 734,
# 747,
# 748,
# 749,
# 751,
# 787,
# 797,
# 801,
# 811,
# 816,
# 818,
# 826,
# 855,
# 872,
# 877,
# 882,
# 884,
# 886,
# 888,
# 891,
# 910,
# 933,
# 937,
# 940,
# 950,
# 951,
# 952,
# 953,
# 968,
# 982,
# 972,
# 980,
# 982,
# 985,
# 6032,
# 6079,
# 6083,
# 6091,
# 6093,
# 6094,
# 6111,
# 6115,
# 6162,
# 6168,
# 6169,
# 6170,
# 6195,
# 6240,
# 6246,
# 6281,
# 6296,
# 6359,
# 6420]


# bad_sample_list[115:118]

[6296, 6359, 6420]

In [57]:
# HPLv2 HEEL
bad_sample_list=[57,
73,
148,
198,
199,
230,
257,
418,
477,
479,
481,
486,
692,
719,
731,
767,
821,
980,
6211]


bad_sample_list[:]

[57,
 73,
 148,
 198,
 199,
 230,
 257,
 418,
 477,
 479,
 481,
 486,
 692,
 719,
 731,
 767,
 821,
 980,
 6211]

In [58]:
print("Length of bad list: {}".format(len(bad_sample_list)))

Length of bad list: 19


In [59]:
bad_sample_list_name = ["image_{:04d}.{}".format(x,i_fmt) for x in bad_sample_list for i_fmt in ['JPG','jpg','png','PNG'] ]
bad_sample_list_name[:5]

['image_0057.JPG',
 'image_0057.jpg',
 'image_0057.png',
 'image_0057.PNG',
 'image_0073.JPG']

In [60]:
len(bad_sample_list_name)

76

In [61]:
path = "train/images/" + 'image_0170.jpg'
print(path)
os.path.isfile(path)

train/images/image_0170.jpg


True

In [62]:
# List Comprehension 
bad_sample_list_name_fmt = [f for f in bad_sample_list_name if os.path.isfile("train/images/" + f)]
bad_sample_list_name_fmt[:5]

['image_0057.png',
 'image_0073.png',
 'image_0148.png',
 'image_0198.jpg',
 'image_0199.jpg']

In [63]:
print("Length of bad list: {}".format(len(bad_sample_list_name_fmt)))

Length of bad list: 19


#### Filter out the bad sample from DF

In [64]:
# # test data frame pop out the bas list
# df_bad_filtered = df[df.images == "image_0001.png"]
# df_bad_filtered

In [65]:
df_good_sample = df_merge[[all([x not in bad_sample_list_name_fmt]) for x in df_merge['images']]]

In [66]:
print([[x not in bad_sample_list_name_fmt] for x in df['images']])

[[True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [False], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [False], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True

In [67]:
# remove x not in bad_sample_list_name_fmt, to show bad name only
print(df_merge[[all([x in bad_sample_list_name_fmt]) for x in df_merge['images']]].images)

56      image_0057.png
72      image_0073.png
147     image_0148.png
197     image_0198.jpg
198     image_0199.jpg
229     image_0230.png
256     image_0257.png
417     image_0418.png
476     image_0477.png
478     image_0479.png
480     image_0481.png
485     image_0486.png
691     image_0692.png
718     image_0719.png
730     image_0731.png
766     image_0767.png
820     image_0821.png
979     image_0980.png
1210    image_6211.jpg
Name: images, dtype: object


### A fun to merge all tasks of bad sample remove

In [68]:
def remove_bad_samples(df):
    BAL = bad_sample_list
    bad_sample_list_name = ["image_{:04d}.{}".format(x,i_fmt) for x in BAL for i_fmt in ['JPG','jpg','png','PNG'] ]
    bad_sample_list_name_fmt = [f for f in bad_sample_list_name if os.path.isfile("train/images/" + f)]
    #df_good_sample =df[[all([x not in bad_sample_list_name_fmt]) for x in df['images']]]
    return df[[all([x not in bad_sample_list_name_fmt]) for x in df['images']]]

In [69]:
test_df_all_good = remove_bad_samples(df_merge)
print("Length of good_df list: {}".format(len(test_df_all_good)))


Length of good_df list: 1405


In [70]:
# df_good_sample =df[[all([x not in bad_sample_list_name_fmt for x in df['images']])]]

In [71]:
"""1424 - 79
= 1345"""
len(df_good_sample)

1405

In [72]:
# check 67
df_good_sample[65:68]

,images,x1,y1,x2,y2
66,image_0067.png,52,155,78,374
67,image_0068.png,53,156,32,372
68,image_0069.png,57,148,85,382


In [ ]:
#check image_0980.png
df_good_sample[949:955]

In [ ]:
#check index for df_good_sample 

df_good_sample.tail(5)

In [ ]:
df_good_sample = df_good_sample.reset_index(drop=True)

In [ ]:
#check index for df_good_sample 

df_good_sample.tail(5)

In [ ]:
print("Length of GOOD list: {}".format(len(df_good_sample)))

## Save to good annotation 'annotation_1345_good.csv'

In [73]:
#1345, annotation_1345_good.csv
# df_good_sample.to_csv(f'annotation_{len(df_good_sample)}_HPL_Good.csv', index=False)

# hpl v2 toe -117= 1307,  annotation_1307_HPLv2Toe_Good.csv
# df_good_sample.to_csv(f'annotation_{len(df_good_sample)}_HPLv2Toe_Good.csv', index=False)

# hpl v2 heel = -19,  annotation_1405_HPLv2Heel_Good.csv
df_good_sample.to_csv(f'annotation_{len(df_good_sample)}_HPLv2Heel_Good.csv', index=False)

In [74]:
!ls -l train/images/image_* | wc -l

1424


In [75]:
!ls -lah annotation_*ood.csv

-rw-r--r-- 1 uu TRI108216 32K Jan 26 14:07 annotation_1123_HPL_Good.csv
-rw-r--r-- 1 uu TRI108216 38K Mar  2 14:11 annotation_1307_HPLv2Toe_Good.csv
-rw-r--r-- 1 uu TRI108216 39K Jan 21 11:14 annotation_1345_good.csv
-rw-r--r-- 1 uu TRI108216 40K Mar  2 14:15 annotation_1405_HPLv2Heel_Good.csv


### Copy all good list to 'train_augm/images' for augment

In [ ]:
!mkdir train_augm/images

In [ ]:
# df_good_list = pd.read_csv('annotation_1345_good.csv')
df_good_list = pd.read_csv('annotation_1424_merge.csv')

In [ ]:
df_good_list.info()

In [ ]:
def copy_good_samples_to_augm(img_name):
    
    train_path = 'train/images/' + img_name
    augm_path = 'train_augm/images/' + img_name   

    shutil.copyfile(train_path, augm_path)

#     print(f'cp {train_path} ---> {augm_path}')


In [ ]:
"""
# Now you can use `progress_apply` instead of `apply`
# and `progress_map` instead of `map`
"""
tqdm.pandas()

In [ ]:
%%time
null_df = df_good_list["images"].progress_map(copy_good_samples_to_augm)

In [ ]:
# !rm -rf train_augm/images/*
!ls -l train_augm/images/image_* | wc -l

## Augmenting training data beforehand

####  Depend on the application, the symmetrical foot pressure map should only apply the flipping and noising, and maybe the shift/offset.

* duplicate images with flip
* name new augmented images with image_6242.png

In [ ]:
# Read full csv
# df_good_list = pd.read_csv('annotation_1345_good.csv')
df_good_list = pd.read_csv('annotation_1424_merge.csv')

In [ ]:
len(df_good_list)

In [ ]:
df_good_list.head(5)

In [ ]:
df_good_list[1340:1346]

### Create the Flip name

In [ ]:
def ConvertToFlop(pathToFile, newpathToFile):
    #yet, the Flop is for horizontal mirroring, flip is or vertical in ImageMagick.
    import subprocess
    p = subprocess.Popen(['convert', '-flop', '-quality', '100', pathToFile, newpathToFile], stdout = subprocess.PIPE, stderr=subprocess.PIPE)
    
    
def augment_flip_map(img_name):
    str_1, str_2 = img_name.split('_',-1)#;print('str_2 : {}'.format(str_2))
    str_3, str_4 = str_2.split('.',-1)#;print('str_2 : {} . {}'.format(str_3,str_4))

    # new fliped name: af-xxx

    augment_flip_name = 'image_' + 'af-' + str(str_3) + '.' + str_4 
#     print('Flip new {} from {}'.format(augment_flip_name, img_name))
    
    # new fliped image: train_augm/images/{augment_flip_name}
    pathToFile = 'train_augm/images/' + img_name
    newpathToFile = 'train_augm/images/' + augment_flip_name
    ConvertToFlop(pathToFile, newpathToFile)
#     print('Flop {} ---> {}'.format(pathToFile, newpathToFile))
    
    return augment_flip_name

In [ ]:
# !rm -rf train_augm/images/image_af-*
!ls -l train_augm/images/image_af-* | wc -l
!ls -l train_augm/images/image_* | wc -l

In [ ]:
# Flip the new image name.

df_aug_flip_list = df_good_list.copy()

df_aug_flip_list["images"] = df_aug_flip_list["images"].progress_map(augment_flip_map)

In [ ]:
len(df_aug_flip_list)

In [ ]:
df_aug_flip_list.head(5)

### Flip the X1,X2 value

In [ ]:
# 55->flip->65 (120-55)

df_aug_flip_list.iloc[:,1] = 120 - df_aug_flip_list.iloc[:,1]


In [ ]:
df_aug_flip_list.iloc[:,3] = 120 - df_aug_flip_list.iloc[:,3]

In [ ]:
# TypeError: '(slice(None, None, None), 1)' is an invalid key
# 這是因為第一列column是標籤，無法處理。這是df不是nparray的差異。
# 請加上iloc，如df.iloc[:,1]。

In [ ]:
df_aug_flip_list.head(5)

In [ ]:
# df_aug_flip_list.to_csv('annotation_dfaug_flip_list.csv', index=False)
df_aug_flip_list.to_csv('annotation_df_1424_aug_flip_list.csv', index=False)

In [ ]:
"""
1000-30bad(this 30bad was not being duplicated.)
424

1394 + 1424
= 2818 in disk
"""
len(df_aug_flip_list)

## Merge train + augm list

In [ ]:
# df_good_list = pd.read_csv('annotation_1345_good.csv')
# df_augm_list = pd.read_csv('annotation_df_aug_flip_list.csv')

df_good_list = pd.read_csv('annotation_1424_merge.csv')
df_augm_list = pd.read_csv('annotation_df_1424_aug_flip_list.csv')


df_traim_augm_list = pd.concat([df_good_list, df_augm_list])

In [ ]:
df_traim_augm_list.info()

In [ ]:
# df_traim_augm_list.to_csv('annotation_2690_augm.csv', index=False)

df_traim_augm_list.to_csv('annotation_2848_augm.csv', index=False)


## Save ALL image with PP plotting for HUMAN-feature engineering

In [ ]:
ann = 'train/annotation.csv'
df = pd.read_csv(ann)

good_df = remove_bad_samples(df)
print("Length of good_df list: {}".format(len(good_df)))

In [ ]:
a6 = 'annotation_a6_df.csv'
a6_df = pd.read_csv(a6)

gooda6_df = pd.concat([good_df, a6_df])
print("Length of gooda6_df list: {}".format(len(gooda6_df)))

In [ ]:
!mkdir train_review/images

In [ ]:
train_dir = 'train/images'
img_name = 'image_6424.jpg'
plt_ud_point_by_path_name(gooda6_df, train_dir, img_name)

In [ ]:
img_index = gooda6_df.loc[gooda6_df['images'] == img_name].index

In [ ]:
print(img_index)

In [ ]:
"""由於合併了先後兩版df造成index重複

img_index = gooda6_df.loc[gooda6_df['images'] == img_name].index
print(gooda6_df.images[img_index])

423    image_0424.png
423    image_6424.jpg
Name: images, dtype: object

使用gooda6_df.reset_index 重新編號

"""
print(gooda6_df.images[img_index])

In [ ]:
gooda6_df[423:424]

In [ ]:
#重新編號
"""
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html

len(gooda6_df_reindex)
	index	images	x1	y1	x2	y2
0	0	image_0001.png	55	136	75	383
1	1	image_0002.png	62	139	42	385
2	2	image_0003.png	47	117	76	368
3	3	image_0004.png	66	119	37	376
4	4	image_0005.png	58	147	85	381

drop=True, 將原本的index刪除，不然會另外加上新的index>
gooda6_df_reindex = gooda6_df.reset_index(drop=True)

	images	x1	y1	x2	y2
0	image_0001.png	55	136	75	383
1	image_0002.png	62	139	42	385
2	image_0003.png	47	117	76	368
3	image_0004.png	66	119	37	376
4	image_0005.png	58	147	85	381
"""
gooda6_df_reindex = gooda6_df.reset_index(drop=True)

In [ ]:
len(gooda6_df_reindex)

In [ ]:
gooda6_df_reindex.head(5)

In [ ]:
img_index = gooda6_df_reindex.loc[gooda6_df_reindex['images'] == img_name].index

In [ ]:
print(img_index)

In [ ]:
print(gooda6_df_reindex.images[img_index])

In [ ]:
train_dir = 'train/images'
img_name = 'image_6109.png'

img_index = gooda6_df_reindex.loc[gooda6_df_reindex['images'] == img_name].index
print('img_index: {}'.format(img_index))

plt_ud_point_by_path_name(gooda6_df_reindex, train_dir, img_name)

In [ ]:
"""
For example:image_06109.png  up[62, 185], down[82, 381]
y1= 85-80 = 105@120
可用的特徵太少 但應該足夠
"""

# test_sample = 'train_augm/images/image_6109.png'
test_sample = 'train_augm/images/image_0202.jpg'

img = tf.io.read_file(test_sample)#can read the byte string paths b'image_0001.png'
img = decode_crop_png_toe(img)
plt.imshow(img)

In [ ]:
test_sample = 'train_augm/images/image_6170.jpg' # p1 almost out of bbox


img = tf.io.read_file(test_sample)#can read the byte string paths b'image_0001.png'
img = decode_crop_png_toe(img)
plt.imshow(img)

In [ ]:
test_sample = 'train_augm/images/image_6004.png'


img = tf.io.read_file(test_sample)#can read the byte string paths b'image_0001.png'
img = decode_crop_png_toe(img)
plt.imshow(img)

In [ ]:
plt_ud_point(gooda6_df_reindex, 1078)

In [ ]:
!mkdir review_tmp

In [ ]:
plt_ud_point_just_save(gooda6_df_reindex, 413)

In [ ]:
for index in range(len(gooda6_df_reindex)):
    plt_ud_point_just_save(gooda6_df_reindex, index)
    print(index)

## some pre test

### Test howto contact the all pred reuslt for submit.CSV

In [ ]:
# append test
aa= [[ 32.0332  , 105.49409 ],
       [ 68.21191 ,  83.02111 ],
       [ 32.07095 ,  99.04422 ]]
bb= [[ 11.0332  , 22.49409 ],
       [ 33.21191 ,  44.02111 ]]
all_pred = np.append(aa, bb, axis=0)#上下接

In [ ]:
all_pred

In [ ]:
all_pred.shape

In [ ]:
all_pred_12 = np.append(all_pred, all_pred, axis=1)#左右接

In [ ]:
all_pred_12

In [ ]:
all_pred_12.shape

### parser a element's value

In [ ]:
# pd index serch by name
# df.set_index("images" , inplace=True) # have to re-set index
# df.loc['image_0001.png'] 


In [ ]:
df_filtered = df[df.images == "image_0001.png"]
df_filtered.head()

### Plot PP image and PP

In [ ]:
def get_img(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def get_img_by_id(df_index):
    img_name = df.iloc[df_index, 0] # [index, table] or [row, columns]
    t_img_p = 'train/images/' + img_name
    return get_img(t_img_p)

    
def what_dtype(data):
    return print(f'dtype: {data.dtype}, shape: {data.shape}')
    
def plt_img(img):
    #plt.figure(figsize=(6,6))
#     plt.title(f'dtype: {img.dtype}')
    plt.axis("off")
    plt.imshow(img)

def plt_points(img, x, y):
    cv2.circle(img, (x,y), 3, (0,0,213), -1)
    
def cvshow_img(img):
    """cvshow_img(img) display imgage with OS's app window not in the notebook."""
    cv2.imshow('cv image', img)
    cv2.waitKey(500)
    cv2.destroyAllWindows()

In [ ]:
def plt_ud_points(df_index):
    img_name = df.iloc[df_index, 0] # [index, table] or [row, columns]

    t_img_p = 'train/images/' + img_name
    img = get_img(t_img_p)
#     plt.figure()
    plt.imshow(img)
    plt.plot(df.x1[df_index], df.y1[df_index], 'r+', markersize=15)
    plt.plot(df.x2[df_index], df.y2[df_index], 'b+', markersize=15)
    plt.axis("off")
    plt.title(img_name)

def plt_ud_points_lines(df_index):
    img_name = df.iloc[df_index, 0] # [index, table] or [row, columns]

    t_img_p = 'train/images/' + img_name
    img = get_img(t_img_p)
#     plt.figure()
    plt.imshow(img)
    plt.plot(df.x1[df_index], df.y1[df_index], 'r+', markersize=15, mew=3)
    plt.plot(df.x2[df_index], df.y2[df_index], 'b+', markersize=15, mew=3)
    # line
    x = [df.x1[df_index], df.x2[df_index]]
    y = [df.y1[df_index], df.y2[df_index]]
    plt.plot(x, y, 'k-', lw=3)
    
    plt.axis("off")
    plt.title(img_name)
    
    
def plt_ud_point(df, df_index):
    img_name = df.iloc[df_index, 0] # [index, table] or [row, columns]

    t_img_p = 'train/images/' + img_name
    img = get_img(t_img_p)
    plt.figure()
    plt.imshow(img)
    plt.plot(df.x1[df_index], df.y1[df_index], 'r+', markersize=15, mew=2.5)
    plt.plot(df.x2[df_index], df.y2[df_index], 'b+', markersize=15, mew=2.5)
    print(f'up[{df.x1[df_index]}, {df.y1[df_index]}], down[{df.x2[df_index]}, {df.y2[df_index]}]')
    plt.axis("off")
    plt.title(img_name) 
    
    
def plt_ud_point_multiple(df, df_index):
    img_name = df.iloc[df_index, 0] # [index, table] or [row, columns]

    t_img_p = 'train/images/' + img_name
    img = get_img(t_img_p)
#     plt.figure() # for multiple image in same figure should be disable this.
    plt.imshow(img)
    plt.plot(df.x1[df_index], df.y1[df_index], 'r+', markersize=25)#, mew=1.5)
    plt.plot(df.x2[df_index], df.y2[df_index], 'r+', markersize=25)#, mew=1.5)
#     print(f'up[{df.x1[df_index]}, {df.y1[df_index]}], down[{df.x2[df_index]}, {df.y2[df_index]}]')
    plt.axis("off")
    plt.title(img_name)
    
    
def plt_ud_point_just_save(df, df_index):
    img_name = df.iloc[df_index, 0] # [index, table] or [row, columns]

    t_img_p = 'train/images/' + img_name
    img = get_img(t_img_p)
    plt.figure()
    plt.imshow(img)
    plt.plot(df.x1[df_index], df.y1[df_index], 'r+', markersize=15, mew=2.5)
    plt.plot(df.x2[df_index], df.y2[df_index], 'b+', markersize=15, mew=2.5)
#     print(f'up[{df.x1[df_index]}, {df.y1[df_index]}], down[{df.x2[df_index]}, {df.y2[df_index]}]')
    plt.axis("off")
    plt.title(img_name, fontsize=8)
    
    review_tmp = 'review_tmp/' + img_name
    plt.savefig(review_tmp,bbox_inches='tight')
    
    
    #不好用 容易弄錯
def plt_ud_point_by_path_name(df, path, img_name): 
    full_img_path = path + '/' + img_name # [index, table] or [row, columns]
#     print(full_img_path)
    
    img = get_img(full_img_path)
    

    #get index of image
    df_index = df.loc[df['images'] == img_name].index
    
    plt.figure()
    plt.imshow(img)
    plt.plot(df.x1[df_index], df.y1[df_index], 'r+', markersize=20, mew=2.5)
    plt.plot(df.x2[df_index], df.y2[df_index], 'r+', markersize=20, mew=2.5)
    print(f'up[{df.x1[df_index]}, {df.y1[df_index]}], down[{df.x2[df_index]}, {df.y2[df_index]}]')
    plt.axis("off")
    plt.title(img_name)    

In [ ]:
# df.reset_index(inplace=True)
plt_ud_point(df,979)

In [ ]:
plt_ud_points_lines(1)

In [ ]:
plt_ud_points_lines(409)

#### Image size, Batch size, toe/heel-offset.

In [ ]:
BATCH_SIZE = 64 #128 #32 #64 #todo: reduce the BS maybe help to reduce the loss
img_height = 120 #512 #224 #100
img_width = 120 #512 #224 #100

y_offset_toe = 80
y_offset_heel = 280 #400-120=280

#### Process path to image tensor in DS

In [ ]:
# im_p = 'train/images/'

# For annotation_revise_aug-f
im_p = 'train_augm/images/'

'''
def get_label(file_path):
    # convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    boolen = parts[-2] == class_names
    #one_hot_num = np.array(boolen, dtype=np.int) not works should use tf.x repalced.
    one_hot_num = tf.dtypes.cast(boolen, tf.int64)
    one_num = tf.argmax(one_hot_num)
    print('one_num:', one_num)
    # Integer encode the label
    return one_num

def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.io.decode_jpeg(img, channels=3)
    # resize the image to the desired size
#     return tf.image.resize(img, [img_height, img_width])# augment 已經resize過一次了 但這邊不先做會比較慢
    return tf.cast(tf.image.resize(img, [img_height, img_width]), tf.uint8)# 避免float over at augment
'''

#
# map list to ds, Toe part.
#

def decode_crop_png_toe(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.io.decode_png(img, channels=3)
    # crop the toe from top-left corner [image, offset_height y1, offset_width x1, target_height, target_width]
    y1=y_offset_toe;    x1=0;    h=img_height;    w=img_width # not the pp location
    img = tf.image.crop_to_bounding_box(img, int(y1), int(x1), h, w)
    #img = tf.image.crop_to_bounding_box(img, int(y1), int(x1), int(y2)-int(y1), int(x2)-int(x1))
    # resize the image to the desired size
    return img

def process_path_toe(file_path,x1,y1,x2,y2):
    file_path = im_p + file_path
    #label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)#can read the byte string paths b'image_0001.png'
    img = decode_crop_png_toe(img)
    return img, [x1,y1-y_offset_toe]

#
# map list to ds, Heel part.
#

def decode_crop_png_heel(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.io.decode_png(img, channels=3)
    # crop the toe from top-left corner [image, offset_height y1, offset_width x1, target_height, target_width]
    y2=y_offset_heel;    x2=0;    h=img_height;    w=img_width # not the pp location
    img = tf.image.crop_to_bounding_box(img, int(y2), int(x2), h, w)
    #img = tf.image.crop_to_bounding_box(img, int(y1), int(x1), int(y2)-int(y1), int(x2)-int(x1))
    # resize the image to the desired size
    return img

def process_path_heel(file_path,x1,y1,x2,y2):
    file_path = im_p + file_path
    #label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)#can read the byte string paths b'image_0001.png'
    img = decode_crop_png_heel(img)
    return img, [x2,y2-y_offset_heel]


#
# test how to put parameters to map
#

def t_ds_map(file_path,x1,y1,x2,y2):
#     img = get_img('train/images/' + str(file_path))
#     print(file_path)
    return file_path,x1,y1,x2,y2 #img, [x1,y1]
    